### Project Description
#### Objective : Address the Challenge with Evaluation and Prediction of Human Mobility Under Extreme Weather Events
#### Datasets : weather_county_2019 | weather_CBG_2019 | social distancing dataset (provided by Safegraph)

##### This notebook is to explore weather_county_2019 Dataset. It will not be used for the data modeling part.

In [0]:
# Packages setup
!pip install geopandas
!pip install plotly
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.functions import col,mean
plt.gcf().clear()
plt.style.use('seaborn-whitegrid')
pd.set_option('display.float_format', lambda x : '%0.3f' %x)
%matplotlib inline


In [0]:
#    This step is to readt the weather_county_2019 dataset. There are one county variable and 6 quantitive variables:
#    1): precip - daily precipitation
#    2): rmax - maximum daily relative humidity
#    3): rmin - minimum daily relative humidity
#    4): srad - surface downwelling solar radiation
#    5): tmax - maximum daily temperature
#    6): tmin - minimum daily temperature
#    7): wind_speed -  wind speed 

weather_county_2019 = spark.sql('''
  select * from default.weather_county_2019
''')
weather_county_2019 = weather_county_2019.withColumn("month", F.month('date'))
weather_county_2019 = weather_county_2019.withColumn("day", F.dayofmonth('date'))
weather_county_2019 = weather_county_2019.withColumn("year", F.year('date'))




# Dataset Info
def spark_df_shape(self):
    return (self.count(),len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_df_shape

print(weather_county_2019.shape())
weather_county_2019.printSchema()
weather_county_2019.select("county").distinct().count()

(1132960, 12)
root
-- county: integer (nullable = true)
-- date: string (nullable = true)
-- precip: double (nullable = true)
-- rmax: double (nullable = true)
-- rmin: double (nullable = true)
-- srad: double (nullable = true)
-- tmin: double (nullable = true)
-- tmax: double (nullable = true)
-- wind_speed: double (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- year: integer (nullable = true)

Out[2]: 3104

In [0]:
#    create monthly table    
weather_county_2019_month = (weather_county_2019.groupBy("county","month")
                            .agg(
                              mean("precip").alias("mean_precip"),
                              mean("rmax").alias("mean_rmax"),
                              mean("rmin").alias("mean_rmin"),
                              mean("srad").alias("mean_srad"),
                              mean("tmin").alias("mean_tmin"),
                              mean("tmax").alias("mean_tmax"),
                              mean("wind_speed").alias("mean_wind_speed"))
                             .sort(col("county").asc(),col("month").desc()))




In [0]:
# Convert the Spark DataFrame back to a pandas DataFrame using Arrow, and display data in panda dataframe
spark.conf.set("spark.sql.execution.arrow.enabled", "true");

weather_county_2019_month_pdf = weather_county_2019_month.select("*").toPandas();

weather_county_2019_month_pdf.head()
weather_county_2019_month_pdf[weather_county_2019_month_pdf.columns[2:]].describe()


mean_precip 
 mean_rmax 
 mean_rmin 
 mean_srad 
 mean_tmin 
 mean_tmax 
 mean_wind_speed 
 
 
 
 
 count 
 37248.000 
 37248.000 
 37248.000 
 37248.000 
 37248.000 
 37248.000 
 37248.000 
 
 
 mean 
 3.240 
 88.853 
 45.796 
 191.136 
 43.911 
 65.416 
 9.153 
 
 
 std 
 2.266 
 8.083 
 9.699 
 76.534 
 18.144 
 19.237 
 1.747 
 
 
 min 
 0.000 
 26.073 
 6.784 
 34.132 
 -16.553 
 3.032 
 3.506 
 
 
 25% 
 1.546 
 85.401 
 41.370 
 117.266 
 30.016 
 51.322 
 7.957 
 
 
 50% 
 2.925 
 90.308 
 46.881 
 194.219 
 44.472 
 67.523 
 9.153 
 
 
 75% 
 4.427 
 94.358 
 51.967 
 260.038 
 59.597 
 81.795 
 10.339 
 
 
 max 
 25.525 
 99.987 
 80.480 
 364.048 
 80.288 
 108.636 
 19.307

In [0]:
#  Note this part of the code was adapted from here -> https://plotly.com/python/choropleth-maps/

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

counties["features"][0]


# note this county is based on county Fips number which assumed to be consistent with the data source
import plotly.express as px
county = weather_county_2019_month_pdf.county.unique()
month = weather_county_2019_month_pdf.month.unique()
fig = px.choropleth(weather_county_2019_month_pdf, geojson=counties, locations='county', color='mean_precip',
                           color_continuous_scale= "viridis",
                           range_color=(0, 5),
                           scope="usa"
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

